## CS/INFO 5304 Assignment 2: Data Visualization <br />

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

### Part A: Interactive Monthly Temperature Visualization

In [ ]:
# Load and preprocess data
weather_df = pd.read_csv('weather.csv', parse_dates=['time'])
weather_df['Ftemp'] = (weather_df['Ktemp'] - 273.15) * 9/5 + 32
weather_df['year'] = weather_df['time'].dt.year
weather_df['month'] = weather_df['time'].dt.month
monthly_avg = weather_df.groupby(['year', 'month'])['Ftemp'].mean().reset_index()

# Create interactive plot
fig = px.line(monthly_avg, x='month', y='Ftemp', 
              animation_frame='year',
              title='Monthly Temperature Trends at Cornell Tech (1950-2021)',
              labels={'Ftemp': 'Temperature (°F)', 'month': 'Month'},
              range_y=[20, 85])

fig.update_layout(
    updatemenus=[dict(type="buttons", direction="right", x=0.1, y=1.15)],
    sliders=[dict(currentvalue={"prefix": "Year: "}, pad={"t": 50})]
)
fig.show()

This visualization shows the monthly average temperatures at Cornell Tech from 1950 to 2021. 

Users can explore how temperature patterns vary across years using an interactive slider.

### Part B: First Year Above 55°F Average

In [34]:
# Calculate annual average temperatures
annual_avg = weather_df.groupby('year')['Ftemp'].mean().reset_index()

# Identify the first year exceeding 55°F
threshold_year = annual_avg[annual_avg['Ftemp'] > 55]['year'].min()
print(f"The first year when the average temperature exceeded 55°F was: {threshold_year}")

# Create visualization
fig = px.line(
    annual_avg, x='year', y='Ftemp',
    title=f'Annual Average Temperature at Cornell Tech (First Year > 55°F: {threshold_year})',
    labels={'year': 'Year', 'Ftemp': 'Temperature (°F)'}
)

# Highlight threshold year
fig.add_vline(
    x=threshold_year,
    line=dict(color='red', dash='dash'),
    annotation_text=f"Threshold Year: {threshold_year}",
    annotation_position="top left"
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
)

fig.show()


The first year when the average temperature exceeded 55°F was: 1953


This graph shows the annual average temperatures at Cornell Tech from 1950 onward, highlighting the first year exceeding 55°F, which was 1953.

### Part C: Interactive Climate Correlation Visualization

In [38]:
# Load NOAA CO2 data
co2_df = pd.read_csv(
    'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_gl.csv', 
    skiprows=38, 
    names=['year', 'co2', 'unc'], 
    engine='python', 
    skipfooter=2
)

# Load NASA global temperature data
temp_df = pd.read_csv(
    'https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv',
    skiprows=1,
    usecols=['Year', 'J-D'],
    na_values=['***', '****', '*****', '******']
).rename(columns={'Year': 'year', 'J-D': 'global_anomaly'})

# Process and merge data
cornell_df = weather_df.groupby('year')['Ftemp'].mean().reset_index()
merged_df = (cornell_df
          .merge(co2_df, on='year')
          .merge(temp_df, on='year')
          .dropna()
          .query('year >= 1979'))  

# Train climate model
model = LinearRegression()
model.fit(merged_df[['co2']], merged_df['Ftemp'])
merged_df['predicted_temp'] = model.predict(merged_df[['co2']])

# Create interactive visualization
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=merged_df['year'], y=merged_df['Ftemp'],
    name='Cornell Temp (°F)',
    mode='lines+markers',
    line=dict(color='#1f77b4'),
    hovertemplate="%{y:.1f}°F in %{x}<extra></extra>"
))

fig.add_trace(go.Scatter(
    x=merged_df['year'], y=merged_df['co2'],
    name='Global CO2 (ppm)',
    yaxis='y2',
    line=dict(color='#2ca02c', dash='dot'),
    hovertemplate="%{y:.1f} ppm CO2 in %{x}<extra></extra>"
))

# Add climate model projection
fig.add_trace(go.Scatter(
    x=merged_df['year'], y=merged_df['predicted_temp'],
    name='CO2-Driven Trend',
    line=dict(color='#ff7f0e', width=3),
    hovertemplate="Model: %{y:.1f}°F in %{x}<extra></extra>"
))

# Add climate milestones
milestones = [
    {'year': 1988, 'text': 'IPCC Established (1988)'},
    {'year': 1997, 'text': 'Kyoto Protocol (1997)'},
    {'year': 2015, 'text': 'Paris Agreement (2015)'}
]

for milestone in milestones:
    fig.add_vline(
        x=milestone['year'], 
        line=dict(color='green', width=1, dash='dash'),
        annotation=dict(text=milestone['text'], xanchor='left')
    )

fig.update_layout(
    title='Cornell Temperature vs Global CO2 (1979-2021)',
    xaxis=dict(
        title='Year',
        rangeslider=dict(visible=True),
        range=[1979, 2021]
    ),
    yaxis=dict(
        title='Temperature (°F)',
        color='blue'
    ),
    yaxis2=dict(
        title='CO2 Concentration (ppm)',
        overlaying='y',
        side='right',
        color='gray'
    ),
    hovermode='x unified',
    template='plotly_white',
    height=600
)

fig.show()

This interactive dashboard integrates Cornell Tech temperature data with global $CO_2$ levels and NASA temperature anomalies, providing insights into climate trends and correlations. It visually communicates the relationship between local temperatures and global climate indicators, including notable policy milestones.